# Conversion from top predictive model to pipelines

I'll be using this script and seeing how easy it is to convert to the pipeline methodology.

https://www.kaggle.com/aharless/xgboost-using-4th-quarter-for-validation/notebook

## Libraries

In [65]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import datetime as dt
from datetime import datetime
import gc
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg

## Setting up the run parameters

In [66]:
runDetails = dict()
modelDesc = dict()

runDetails['MAKE_SUBMISSION'] = True          # Generate output file.
runDetails['CV_ONLY'] = False                 # Do validation only; do not generate predicitons.
runDetails['FIT_FULL_TRAIN_SET'] = True       # Fit model to full training set after doing validation.
runDetails['FIT_2017_TRAIN_SET'] = False      # Use 2017 training data for full fit (no leak correction)
runDetails['USE_SEASONAL_FEATURES'] = True
runDetails['VAL_SPLIT_DATE'] = '2016-09-15'   # Cutoff date for validation split
runDetails['FUDGE_FACTOR_SCALEDOWN'] = 0.3    # exponent to reduce optimized fudge factor for prediction
runDetails['OPTIMIZE_FUDGE_FACTOR'] = True    # Optimize factor by which to multiply predictions.

modelDesc['LEARNING_RATE'] = 0.007           # shrinkage rate for boosting roudns
modelDesc['ROUNDS_PER_ETA'] = 20             # maximum number of boosting rounds times learning rate



xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': modelDesc['LEARNING_RATE'],
    'max_depth': 7, 
    'subsample': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 5.0,
    'alpha': 0.65,
    'colsample_bytree': 0.5,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

## Reading in data and encoding objects to int

In [67]:
import os

os.chdir('C:/Users/Evan/Documents/GitHub/Zillow/data')

properties = pd.read_csv('properties_2016.csv', low_memory = False)
properties17 = pd.read_csv('properties_2017.csv', low_memory = False)

train = pd.read_csv("train_2016_v2.csv", low_memory = False)
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

## Creating agg features

In [68]:
# Number of properties in the zip
zip_count = properties['regionidzip'].value_counts().to_dict()

# Number of properties in the city
city_count = properties['regionidcity'].value_counts().to_dict()

# Median year of construction by neighborhood
medyear = properties.groupby('regionidneighborhood')['yearbuilt'].aggregate('median').to_dict()

# Mean square feet by neighborhood
meanarea = properties.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()

# Neighborhood latitude and longitude
medlat = properties.groupby('regionidneighborhood')['latitude'].aggregate('median').to_dict()
medlong = properties.groupby('regionidneighborhood')['longitude'].aggregate('median').to_dict()

In [69]:
train_df = train.merge(properties, how='left', on='parcelid')
select_qtr4 = pd.to_datetime(train_df["transactiondate"]) >= runDetails['VAL_SPLIT_DATE']
if runDetails['USE_SEASONAL_FEATURES']:
    basedate = pd.to_datetime('2015-11-15').toordinal()

In [70]:
del train
gc.collect()

1169

In [71]:
# Inputs to features that depend on target variable
# (Ideally these should be recalculated, and the dependent features recalculated,
#  when fitting to the full training set.  But I haven't implemented that yet.)

# Standard deviation of target value for properties in the city/zip/neighborhood
citystd = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("std").to_dict()
zipstd = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("std").to_dict()
hoodstd = train_df[~select_qtr4].groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()

In [87]:
def calculate_features(df):
    
    # Nikunj's features
    # Number of properties in the zip
    df['N-zip_count'] = df['regionidzip'].map(zip_count)
    # Number of properties in the city
    df['N-city_count'] = df['regionidcity'].map(city_count)
    # Does property have a garage, pool or hot tub and AC?
    df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
                         (df['pooltypeid10']>0) & \
                         (df['airconditioningtypeid']!=5))*1 

    # More features
    # Mean square feet of neighborhood properties
    df['mean_area'] = df['regionidneighborhood'].map(meanarea)
    # Median year of construction of neighborhood properties
    df['med_year'] = df['regionidneighborhood'].map(medyear)
    # Neighborhood latitude and longitude
    df['med_lat'] = df['regionidneighborhood'].map(medlat)
    df['med_long'] = df['regionidneighborhood'].map(medlong)

    df['zip_std'] = df['regionidzip'].map(zipstd)
    df['city_std'] = df['regionidcity'].map(citystd)
    df['hood_std'] = df['regionidneighborhood'].map(hoodstd)
    
    if runDetails['USE_SEASONAL_FEATURES']:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)
        
    return(df)
        
def calculate_features_new(df):
    
    # Nikunj's features
    # Number of properties in the zip
    df['N-zip_count'] = df['regionidzip'].agg('count')
    # Number of properties in the city
    df['N-city_count'] = df['regionidcity'].agg('count')
    # Does property have a garage, pool or hot tub and AC?
    df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
                         (df['pooltypeid10']>0) & \
                         (df['airconditioningtypeid']!=5))*1 

    # More features
    # Mean square feet of neighborhood properties
    df['mean_area'] = df['regionidneighborhood'].agg('median')
    # Median year of construction of neighborhood properties
    df['med_year'] = df['regionidneighborhood'].agg('median')
    # Neighborhood latitude and longitude
    df['med_lat'] = df['regionidneighborhood'].agg('median')
    df['med_long'] = df['regionidneighborhood'].agg('median')

    df['zip_std'] = df['regionidzip'].map(zipstd)
    df['city_std'] = df['regionidcity'].map(citystd)
    df['hood_std'] = df['regionidneighborhood'].map(hoodstd)
    
    if runDetails['USE_SEASONAL_FEATURES']:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)  
        
    return(df)

In [88]:
df = calculate_features(train_df)
dfTest = calculate_features_new(train_df)

df.equals(dfTest)

True

In [34]:
dropvars = ['parcelid', 'airconditioningtypeid', 'buildingclasstypeid',
            'buildingqualitytypeid', 'regionidcity']
droptrain = ['logerror', 'transactiondate']

In [35]:
calculate_features(train_df)

x_valid = train_df.drop(dropvars+droptrain, axis=1)[select_qtr4]
y_valid = train_df["logerror"].values.astype(np.float32)[select_qtr4]

print('Shape full training set: {}'.format(train_df.shape))
print('Dropped vars: {}'.format(len(dropvars+droptrain)))
print('Shape valid X: {}'.format(x_valid.shape))
print('Shape valid y: {}'.format(y_valid.shape))

train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
print('\nFull training set after removing outliers, before dropping vars:')     
print('Shape training set: {}\n'.format(train_df.shape))

if runDetails['FIT_FULL_TRAIN_SET']:
    full_train = train_df.copy()

train_df=train_df[~select_qtr4]
x_train=train_df.drop(dropvars+droptrain, axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)
n_train = x_train.shape[0]
print('Training subset after removing outliers:')     
print('Shape train X: {}'.format(x_train.shape))
print('Shape train y: {}'.format(y_train.shape))

if runDetails['FIT_FULL_TRAIN_SET']:
    x_full = full_train.drop(dropvars+droptrain, axis=1)
    y_full = full_train["logerror"].values.astype(np.float32)
    n_full = x_full.shape[0]
    print('\nFull trainng set:')     
    print('Shape train X: {}'.format(x_train.shape))
    print('Shape train y: {}'.format(y_train.shape))

Shape full training set: (90275, 72)
Dropped vars: 7
Shape valid X: (14304, 65)
Shape valid y: (14304,)

Full training set after removing outliers, before dropping vars:
Shape training set: (88528, 72)

Training subset after removing outliers:
Shape train X: (74478, 65)
Shape train y: (74478,)

Full trainng set:
Shape train X: (74478, 65)
Shape train y: (74478,)


C:\Users\Evan\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [42]:
if not runDetails['CV_ONLY']:
    test_df = properties.copy()
    droptest = []
    if runDetails['USE_SEASONAL_FEATURES']:
        test_df['transactiondate'] = '2016-10-31'
        droptest = ['transactiondate']
    calculate_features(test_df)
    x_test = test_df.drop(dropvars+droptest, axis=1)
    print('Shape test: {}'.format(x_test.shape))

    # Process properties for 2017
    properties = properties17
    for c in properties.columns:
        properties[c]=properties[c].fillna(-1)
        if properties[c].dtype == 'object':
            lbl = LabelEncoder()
            lbl.fit(list(properties[c].values))
            properties[c] = lbl.transform(list(properties[c].values))
    zip_count = properties['regionidzip'].value_counts().to_dict()
    city_count = properties['regionidcity'].value_counts().to_dict()
    medyear = properties.groupby('regionidneighborhood')['yearbuilt'].aggregate('median').to_dict()
    meanarea = properties.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()
    medlat = properties.groupby('regionidneighborhood')['latitude'].aggregate('median').to_dict()
    medlong = properties.groupby('regionidneighborhood')['longitude'].aggregate('median').to_dict()

    test_df = properties.copy()
    if runDetails['USE_SEASONAL_FEATURES']:
        test_df['transactiondate'] = '2017-10-31'
    calculate_features(test_df)
    x_test17 = test_df.drop(dropvars+droptest, axis=1)   
    del test_df

Shape test: (2985217, 65)


In [37]:
del train_df
del select_qtr4
gc.collect()

28

## Getting the data into a xgboost friendly format

In [43]:
dtrain = xgb.DMatrix(x_train, y_train)
dvalid_x = xgb.DMatrix(x_valid)
dvalid_xy = xgb.DMatrix(x_valid, y_valid)
if not runDetails['CV_ONLY']:
    dtest = xgb.DMatrix(x_test)
    dtest17 = xgb.DMatrix(x_test17)
    del x_test

In [44]:
del x_train
gc.collect()

224

In [47]:
num_boost_rounds = round( modelDesc['ROUNDS_PER_ETA'] / xgb_params['eta'] )
early_stopping_rounds = round( num_boost_rounds / 20 )
print('Boosting rounds: {}'.format(num_boost_rounds))
print('Early stoping rounds: {}'.format(early_stopping_rounds))

Boosting rounds: 2857
Early stoping rounds: 143


In [50]:
evals = [(dtrain,'train'),(dvalid_xy,'eval')]
model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds,
                  evals=evals, early_stopping_rounds=early_stopping_rounds, 
                  verbose_eval=10)

[0]	train-mae:0.053445	eval-mae:0.065273
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 143 rounds.
[10]	train-mae:0.053357	eval-mae:0.065206
[20]	train-mae:0.053269	eval-mae:0.065138
[30]	train-mae:0.053188	eval-mae:0.065083
[40]	train-mae:0.053106	eval-mae:0.065026
[50]	train-mae:0.053027	eval-mae:0.064976
[60]	train-mae:0.052953	eval-mae:0.064925
[70]	train-mae:0.052887	eval-mae:0.064881
[80]	train-mae:0.052826	eval-mae:0.064841
[90]	train-mae:0.052769	eval-mae:0.064803
[100]	train-mae:0.052722	eval-mae:0.064775
[110]	train-mae:0.052665	eval-mae:0.064742
[120]	train-mae:0.052614	eval-mae:0.064714
[130]	train-mae:0.052567	eval-mae:0.064689
[140]	train-mae:0.052523	eval-mae:0.064663
[150]	train-mae:0.052475	eval-mae:0.064637
[160]	train-mae:0.052428	eval-mae:0.064613
[170]	train-mae:0.052383	eval-mae:0.064592
[180]	train-mae:0.052342	eval-mae:0.064572
[190]	train-mae:0.052307	eval-mae:0.064555
[200]	tra

In [52]:
valid_pred = model.predict(dvalid_x, ntree_limit=model.best_ntree_limit)
print( "XGBoost validation set predictions:" )
print( pd.DataFrame(valid_pred).head() )
print("\nMean absolute validation error:")
mean_absolute_error(y_valid, valid_pred)

XGBoost validation set predictions:
          0
0  0.001607
1  0.022175
2  0.020656
3  0.012385
4  0.019222

Mean absolute validation error:


0.064258203

In [58]:
if runDetails['OPTIMIZE_FUDGE_FACTOR']:
    mod = QuantReg(y_valid, valid_pred)
    res = mod.fit(q=.5)
    print("\nLAD Fit for Fudge Factor:")
    print(res.summary())

    fudge = res.params[0]
    print("Optimized fudge factor:", fudge)
    print("\nMean absolute validation error with optimized fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))

    fudge **= runDetails['FUDGE_FACTOR_SCALEDOWN']
    print("Scaled down fudge factor:", fudge)
    print("\nMean absolute validation error with scaled down fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))
else:
    fudge=1.0


LAD Fit for Fudge Factor:
                         QuantReg Regression Results                          
Dep. Variable:                      y   Pseudo R-squared:              0.01331
Model:                       QuantReg   Bandwidth:                    0.009325
Method:                 Least Squares   Sparsity:                       0.1024
Date:                Sat, 07 Oct 2017   No. Observations:                14304
Time:                        15:18:38   Df Residuals:                    14303
                                        Df Model:                            1
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9598      0.027     35.599      0.000       0.907       1.013
Optimized fudge factor: 0.959837

Mean absolute validation error with optimized fudge factor: 
0.064255
Scaled down fudge factor: 0.987777771561

Mean absolute validation error with s

In [59]:
if runDetails['FIT_FULL_TRAIN_SET'] and not runDetails['CV_ONLY']:
    if runDetails['FIT_2017_TRAIN_SET']:
        
        train = pd.read_csv('train_2017.csv')
        train_df = train.merge(properties, how='left', on='parcelid')
        citystd = train_df.groupby('regionidcity')['logerror'].aggregate("std").to_dict()
        zipstd = train_df.groupby('regionidzip')['logerror'].aggregate("std").to_dict()
        hoodstd = train_df.groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()
        calculate_features(train_df)
        x_full = train_df.drop(dropvars+droptrain, axis=1)
        y_full = train_df["logerror"].values.astype(np.float32)
        n_full = x_full.shape[0]     
        
    dtrain = xgb.DMatrix(x_full, y_full)
    num_boost_rounds = int(model.best_ntree_limit*n_full/n_train)
    full_model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds, 
                           evals=[(dtrain,'train')], verbose_eval=10)

[0]	train-mae:0.053266
[10]	train-mae:0.053176
[20]	train-mae:0.053092
[30]	train-mae:0.053007
[40]	train-mae:0.052932
[50]	train-mae:0.052855
[60]	train-mae:0.052788
[70]	train-mae:0.052725
[80]	train-mae:0.052671
[90]	train-mae:0.052615
[100]	train-mae:0.052562
[110]	train-mae:0.05251
[120]	train-mae:0.052461
[130]	train-mae:0.052414
[140]	train-mae:0.052365
[150]	train-mae:0.052319
[160]	train-mae:0.05228
[170]	train-mae:0.052237
[180]	train-mae:0.052199
[190]	train-mae:0.052164
[200]	train-mae:0.052128
[210]	train-mae:0.052095
[220]	train-mae:0.052058
[230]	train-mae:0.052024
[240]	train-mae:0.051993
[250]	train-mae:0.051962
[260]	train-mae:0.051933
[270]	train-mae:0.051905
[280]	train-mae:0.051878
[290]	train-mae:0.051849
[300]	train-mae:0.051818
[310]	train-mae:0.051792
[320]	train-mae:0.051766
[330]	train-mae:0.051741
[340]	train-mae:0.051716
[350]	train-mae:0.051689
[360]	train-mae:0.051663
[370]	train-mae:0.051638
[380]	train-mae:0.051614
[390]	train-mae:0.051592
[400]	train-m